In [53]:
import pandas as pd
import folium as fl
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import math
import json

# Análisis de Datos geográficos - Bogotá (Sector SEGURIDAD)  
**Carga de Archivos**

In [54]:
data = json.load(open('datos/prueba_mod.json' , encoding="utf8"))
#df = pd.DataFrame(data["features"])
df = pd.DataFrame(data["features"])

In [55]:
df

,NGeClasifi,NGeNombre,coordinates
0,TRANS,Estación de La Sabana,"[-74.0832351919999, 4.607221417000062]"
1,TRANS,Estación de Usaquén,"[-74.0378237459999, 4.690720078000084]"
2,TRANS,Terminal de Transportes de Bogotá,"[-74.11548348799994, 4.653418440000053]"
3,TRANS,Terminal de Transportes Del Sur,"[-74.17638567599994, 4.597639136000055]"
4,TRANS,Terminal de Pasajeros de Transporte Interurban...,"[-74.04109375899992, 4.769708439000055]"
...,...,...,...
530,SALUD,Hospital San Blas E.S.E. - Institución de II N...,"[-74.08364775599995, 4.570079443000054]"
531,SALUD,Hospital San Cristóbal E.S.E. - Institución de...,"[-74.09360697599993, 4.575143589000049]"
532,SALUD,Hospital Vista Hermosa E.S.E. Institución de I...,"[-74.14464842899991, 4.552677168000059]"
533,SALUD,Hospital Tunjuelito E.S.E. - Institución de II...,"[-74.12597812799993, 4.559092804000045]"


In [56]:
df['cord_ok'] = [[y,x] for x,y in df['coordinates']]
df['color'] = ['green' if x == 'SEG-JUS' else 'blue'  if x == 'SALUD' else 'red' for x in df['NGeClasifi'] ]

In [57]:
points_coords_list=[i for i in df.cord_ok.tolist()]
points_colors=df.color.tolist()

In [61]:
df_seg=df[df['color']== 'green'].reset_index()
points_coords_list_seg=[i for i in df_seg.cord_ok.tolist()]

In [62]:
df_od = pd.read_csv('datos/viajes_geo.csv', sep=';' )

In [70]:
def pintar_mapa(df, col_coord, col_pct, color) :
    default_map_loc=[4.638255, -74.085976]
    map_recorrido = fl.Map(location=default_map_loc, zoom_start=12, tiles='cartodbpositron')

    points=[eval(i) for i in df[col_coord].tolist()]
    
    for i in range(0,len(points)):
        fl.CircleMarker(location=points[i],
                        popup=str(df[col_pct].tolist()[i]*100)+'%',
                        radius=df[col_pct].tolist()[i] *1000, 
                        color=color).add_to(map_recorrido)

    for i in range(0,len(points_coords_list_salud)):
        #fl.Marker(points_coords_list_salud[i],popup=str(df_salud.NGeNombre[i]),icon=fl.Icon(color=points_colors_salud[i], icon='plus-square', prefix='fa' )).add_to(map_salud)
        fl.CircleMarker(location=points_coords_list_salud[i],popup=str(df_salud.NGeNombre[i]), radius=1, color='green').add_to(map_recorrido)

    return map_recorrido

**Sector Seguridad, estratos 2,3 y 4. Que solo van para el trabajo en medios diferentes a pie, bici y patineta  
Orígenes**

In [71]:
#Filtro por sector salud, estratos 2,3 y 4. Que solo van para el trabajo en medios diferentes a pie, bici y patineta 
df_od_seg = df_od[(df_od['p7_id_actividad_economica']==15) & 
                    (df_od['p5_estrato'].isin([1,2,3,4]))  & 
                    (df_od['p17_Id_motivo_viaje'] != 'VOLVER CASA') & 
                    ~(df_od['modo_principal'].isin(['A pie','Bicicleta', 'Patineta']))]
df_od_seg_grp=df_od_seg.groupby(['coordenadas_origen'])['id_viaje'].count().reset_index()
df_od_seg_grp['pct_viajes'] = (df_od_seg_grp['id_viaje']/len(df_od_seg)) 

In [72]:
pintar_mapa(df_od_seg_grp,'coordenadas_origen', 'pct_viajes','orange')

- Repartidos  

**Destinos**

In [74]:
df_od_seg_grp=df_od_salud.groupby(['coordenadas_destino'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))
pintar_mapa(df_od_salud_grp,'coordenadas_destino', 'pct_viajes','purple')

- Destinos más comunes: Teusaquillo, Chapinero y Usaquén  

### Análisis Sector Seguridad por Franja Horaria - MAÑANA 
**Orígenes**

In [75]:
def get_hora(hora) :
    return int(hora.split(':')[0])
df_od['hora_inicio'] = df_od['hora_inicio_viaje'].apply(get_hora)
df_od['hora_fin'] = df_od['p31_hora_llegada'].apply(get_hora)

In [77]:
df_od_salud = df_od[(df_od['p7_id_actividad_economica']==15) & 
                    (df_od['p5_estrato'].isin([1,2,3,4]))  & 
                    ~(df_od['modo_principal'].isin(['A pie','Bicicleta', 'Patineta'])) &
                   (df_od['hora_inicio'].isin([4,5,6,7,8,9,10]))]
df_od_salud_grp=df_od_salud.groupby(['coordenadas_origen'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.005], 'coordenadas_origen', 'pct_viajes', 'orange')

- Repartidos  

**Destinos**

In [78]:
df_od_salud_grp=df_od_salud.groupby(['coordenadas_destino'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.01], 'coordenadas_destino', 'pct_viajes', 'purple')

- Destinos más comunes: Teusaquillo, Chapinero y Usaquén  

### Análisis Sector Salud por Franja Horaria - TARDE
**Orígenes**

In [79]:
df_od_salud = df_od[(df_od['p7_id_actividad_economica']==15) & 
                    (df_od['p5_estrato'].isin([1,2,3,4]))  & 
                    ~(df_od['modo_principal'].isin(['A pie','Bicicleta', 'Patineta'])) &
                   (df_od['hora_inicio'].isin([11,12,13,14,15,16]))]
df_od_salud_grp=df_od_salud.groupby(['coordenadas_origen'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.01], 'coordenadas_origen', 'pct_viajes', 'orange')

- Coincide con Destinode la mañana

**Destinos**

In [80]:
df_od_salud_grp=df_od_salud.groupby(['coordenadas_destino'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.01], 'coordenadas_destino', 'pct_viajes', 'purple')

- No vuelven a la casa? Algunos van a Chía

### Análisis Sector Salud por Franja Horaria - NOCHE 
**Orígenes**

In [81]:
df_od_salud = df_od[(df_od['p7_id_actividad_economica']==15) & 
                    (df_od['p5_estrato'].isin([1,2,3,4]))  & 
                    ~(df_od['modo_principal'].isin(['A pie','Bicicleta', 'Patineta'])) &
                   (df_od['hora_inicio'].isin([0,1,2,3,17,18,19,20,21,22,23]))]
df_od_salud_grp=df_od_salud.groupby(['coordenadas_origen'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.01], 'coordenadas_origen', 'pct_viajes', 'orange')

- Coindice con ubicaciones hospitals destino en la mañana  

**Destinos**

In [82]:
df_od_salud_grp=df_od_salud.groupby(['coordenadas_destino'])['id_viaje'].count().reset_index()
df_od_salud_grp['pct_viajes'] = (df_od_salud_grp['id_viaje']/len(df_od_salud))

pintar_mapa(df_od_salud_grp[df_od_salud_grp['pct_viajes']>0.01], 'coordenadas_destino', 'pct_viajes', 'purple')

- Regresan al Norte, Sopó y Chía